In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from src.model.modeling_krop import Krop
from src.tokenizer.tokenizer import KropTokenizer
from src.data.dataset import get_dacon_train_dataloader, get_dacon_dev

In [3]:
DATASET_NAME = 'jwengr/DACON-Korean-Review-Obfuscation'
BATCH_SIZE=2
BASE_MODEL_NAME='Qwen/Qwen3-0.6B'

In [4]:
train_dl = get_dacon_train_dataloader(DATASET_NAME, batch_size=BATCH_SIZE)
dev_dl, test_dl = get_dacon_dev_test_dataloader(DATASET_NAME, batch_size=BATCH_SIZE)

In [5]:
krop_tokenizer = KropTokenizer(base_tokenizer_name=BASE_MODEL_NAME)
krop = Krop(base_model_name=BASE_MODEL_NAME)
krop.set_tokenizer(krop_tokenizer)

In [6]:
for batch in train_dl:
    break

In [13]:
loss ,logits, pred_ids, sentence_denoised = krop.forward(batch['sentence'], batch['sentence_noisy'], task='char', pred=True)

In [12]:
batch

{'sentence': ['퓨갸 녀뮤 좋야용! 짤 뇰코 캅님따!!',
  '쳬갑 친냔 1월인갼? 2월춤웨 캅봤는톄, 67쯩엣써 슉쏢룰 잡앗셔오. 껴씰톡 클렇교 팡됴 엶쩡 넓꾜, 씹섧또 깩꿋햐교 부까 진쨔 끝넥줍뉘댜. 굵립꼬 억퀵갸 윌폰 슥쉼맞섦뭬셧 낢싣갸 좋우먼 뵤인닳꼬 하눼옴. 윕엑 100쭝엣 쓰탸뻑습또 졍맑 좋교, 헤욀옇행 깐눈 겄 까뜬 눅뀌뮈 뜨렁써오. 큰떼 쌀착 았쒸운 겡 옐씨튐 뮈떼 샹갸뚤뤼 꺼읠 공쉬릴예옷ㅠㅠ 굻련뎨 웠텨퍅끄됴 엶청 꾸케 잊덞람꾜욘. 삯윤났동 잇코, 엎웨 햇윤때 헷쑵욕쟝또 잇썽섟 청맒 좋야써옷. 굻릴교 웰싶틴옛셔 푿싼 룟뎨얼튼 뇰익귁쿡카 뵤잎뎌라곰요.'],
 'sentence_noisy': ['뷰가 너무 좋아요! 잘 놀고 갑니다!!',
  '제가 지난 1월인가? 2월쯤에 가봤는데, 67층에서 숙소를 잡았어요. 거실도 그렇고 방도 엄청 넓고, 시설도 깨끗하고 뷰가 진짜 끝내줍니다. 그리고 여기가 일본 스시마섬에서 날씨가 좋으면 보인다고 하네요. 위에 100층에 스타벅스도 정말 좋고, 해외여행 가는 것 같은 느낌이 들었어요. 근데 살짝 아쉬운 게 엘시티 밑에 상가들이 거의 공실이에요ㅠㅠ 그런데 워터파크도 엄청 크게 있더라고요. 사우나도 있고, 옆에 해운대 해수욕장도 있어서 정말 좋았어요. 그리고 엘시티에서 부산 롯데월드 놀이기구가 보이더라고요.']}

In [14]:
sentence_denoised

['됸뛧!쵧칧!튋녈쪉!!햋!맀녤!볍튥칟!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '뜃껻!뻘쨄!!쾗츻킥!!!뭻긿렷!눻륄뗐겴!!!!믾턬궐!웄셓쫘!턻츙퉑싶!!우촢럿!껬꿏엜!규룗!뵨뜅!뵙웄!!뜗섗혖!뵗햬맹웑!쨬탻!뙑섓!턓늣큄쾄룓!!칙퉗톺!퓽밨룗!윱섓!쨣옼퍙높뉍셤!벘풗퉑!뗑뜓켇!퍣펮꽓껮!줫팙뜙!!좔쉡!!!!뵧헭!먭퉧륤툓꽣!푯짊!닓턙!!쯩븡덣뵣!껙좇!뼣!킷턤!헭뙙툣!큒섓궤케!!떙슴!낔젋!뗤떭뗙!빣!앍뭃옫!턤껣!덙뜄렝뵣!끣먤!땣즓꾒섙녜!!!떙끭뺤!쌜뼓엾왜뙓!씐흴!즧쉥!띭뭙듷럭펏!!턮뚀감꽣!휈홂!!뼙꾤!섪뵓퓁!셓젒먈팣썻!뜻왙뗤!튀팓!엷퍒쉥뙧!!툘꾤샙!맞먮떓퉑뭤!띷끮!뭗큫낑옄!짫덒툈쨪숵!눪뮘팣뗀칣뚓!']